У попередньому завданні вам потрібно було створити:

- карту, котра відображає рівень приросту / скорочення населення за регіонами станом на останній доступний рік;

- лінійний графік, котрий відображає рівень приросту / скорочення населення за регіонами за всіма роками (одна лінія на графіку - один регіон).

Для цього завдання вам потрібно налагодити інтеракцію між картою та графіком.

При наведенні на якийсь регіон на карті він має виділятися на графіку за допомогою товщини лінії та її кольору.

При наведенні на якусь лінію на графіку відповідний їй регіон має виділятися на карті за допомогою прозорості.

In [1]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [2]:
df = pd.read_csv('population_trends.csv')
df = df[df.region != 'Ukraine']
df.head()

,region,year,rate
31,Crimea,1989,3.9
32,Crimea,1990,2.5
33,Crimea,1991,0.9
34,Crimea,1992,-0.7
35,Crimea,1993,-2.7


In [3]:
ukraine_map = gpd.read_file('ukraine.json')
ukraine_map = ukraine_map[['NAME_1', 'geometry']]
ukraine_map = ukraine_map.rename(columns={'NAME_1': 'region'})
ukraine_map.head()

,region,geometry
0,Cherkasy,"MULTIPOLYGON (((31.32614 48.74507, 31.31716 48..."
1,Chernihiv,"MULTIPOLYGON (((33.09283 50.50966, 33.09261 50..."
2,Chernivtsi,"MULTIPOLYGON (((24.93280 47.72794, 24.93301 47..."
3,Crimea,"MULTIPOLYGON (((33.79291 44.39153, 33.79465 44..."
4,Dnipropetrovs'k,"MULTIPOLYGON (((33.93176 47.48407, 33.92332 47..."


In [4]:
df2019 = df.loc[df.year == 2019]
df2019.head()

,region,year,rate
61,Crimea,2019,NaN
92,Vinnytsya,2019,-7.9
123,Volyn,2019,-2.8
154,Dnipropetrovs'k,2019,-8.9
185,Donets'k,2019,NaN


In [5]:
select_region = alt.selection_single(on = 'mouseover', fields = ['region'], nearest = False, empty = 'all')
select_line = alt.selection_single(on = 'mouseover', fields = ['region'], nearest = False, empty = 'none')

In [6]:
line_chart = alt.Chart(df).mark_line().encode(
    x = alt.X('year:Q',
              axis = alt.Axis(format = "1000f")),
    y = alt.Y('rate:Q'),
    detail = alt.Detail('region:N'),
    
    color=alt.condition(
        select_line,
        alt.value("#d60020"),
        alt.value("#a3a3a3")),
    
    strokeWidth = alt.condition(
    select_line,
    alt.value(3),
    alt.value(1)
    ),
    tooltip = alt.Tooltip('region:N')
)

line_chart = line_chart.add_selection(select_region).properties(title = "Рівень приросту / скорочення населення за регіонами за всіма роками", 
                                                                width = 800, height = 500)

In [23]:
line_chart.show()

Displaying chart at http://localhost:16242/


In [7]:
basee = alt.Chart(ukraine_map).mark_geoshape(fill = 'lightgrey', stroke = 'grey').encode(tooltip = [alt.Tooltip('region:N')])
base = basee.properties(width = 800, height = 500)

In [8]:
ukraine = alt.Chart(ukraine_map).transform_lookup(
    lookup = 'region',
    from_ = alt.LookupData(data = df2019, 
                           key = 'region',
                           fields=['region', 'rate'])
).mark_geoshape(stroke = 'grey').encode(
    color = alt.Color('rate:Q', 
                      scale = alt.Scale(scheme = 'reds', reverse=True), 
                      title='Rate',
                      legend=alt.Legend(
                            orient = 'bottom-left',
                            gradientLength = 250
                            )),
    tooltip = [
        alt.Tooltip('region:N'),
        alt.Tooltip('rate:N')],
    
    opacity = alt.condition(
        select_region,
        alt.value(1),
        alt.value(0.2)
    )
)

rate_map = alt.layer(base, ukraine)
rate_map = rate_map.add_selection(select_line).properties(title = "Рівень приросту / скорочення населення за регіонами станом на 2019 рік",
                                                          width = 800, height = 500)


In [13]:
rate_map.show()

Displaying chart at http://localhost:16242/


In [9]:
final_chart = alt.hconcat(rate_map.properties(width = 800, height = 500), 
            line_chart.properties(width = 800, height = 500)
                         ).properties(background = '#F9F9F9', padding = 25).configure_concat(spacing = 0)

In [10]:
final_chart.show()

Displaying chart at http://localhost:17721/
